<a href="https://colab.research.google.com/github/joony0512/short_codes/blob/main/Word_Crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-docx


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 83.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184491 sha256=2d260f26f25c363bd6b5a3b19d1a5fb2288bc7a775ab7cde30514a64338b1fe2
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx


# 페이지 하나

In [ ]:
import pandas as pd
import re

try:
    from xml.etree.cElementTree import XML
except ImportError:
    from xml.etree.ElementTree import XML
import zipfile

NAMESPACE = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}'
PARA = NAMESPACE + 'p'
TEXT = NAMESPACE + 't'

def get_docx_text(filename):
    document = zipfile.ZipFile(filename)
    xml_content = document.read('word/document.xml')
    document.close()
    tree = XML(xml_content)
    paragraphs = []
    for paragraph in tree.iter(PARA):
        texts = [node.text
                for node in paragraph.iter(TEXT)
                if node.text]
        if texts:
            paragraphs.append(''.join(texts))
    return '\n\n'.join(paragraphs)
    
def extract_text_to_dataframe(filename):
    extracted_text = get_docx_text(filename)
    paragraphs = extracted_text.split('\n\n')
    table_data = []
    for paragraph in paragraphs:
        row_data = paragraph.split('\n')
        table_data.append(row_data)
    df = pd.DataFrame(table_data)
    return df

def extract_location(df):
    location = df[df.astype(str).apply(lambda x: x.str.contains('소재지', na=False)).any(axis=1)].iloc[:, 0]
    location = location.str.extract(r'([가-힣\s]+)').values.flatten()
    location = [loc.strip() for loc in location if loc.strip() != '소재지']
    location = ' '.join(location)
    return location

# 추출된 데이터 프레임 생성
extracted_text = extract_text_to_dataframe('/content/drive/MyDrive/노동연구원/IR_KOR.docx')

# 소재지 추출
location = extract_location(extracted_text)

# "소재지" 키워드 제거
location = location.replace('소재지', '')
location = location.replace('설립일', '')
location = location.replace(' ', '')



text = get_docx_text('/content/drive/MyDrive/노동연구원/IR_KOR.docx')


# 대표자 (생년) 추출
# 대표자 이름과 생년 추출
founder_pattern = r"대표자 \(생년\)\s*(\S+)\s*\((\d{4})년\)"
founder_match = re.search(founder_pattern, text)
if founder_match:
    founder_name = founder_match.group(1)
    founder = founder_match.group(2)
else:
    founder_name = None
    founder= None


# 사업화 단계 추출
business_stage = re.findall(r"시리즈\s*([A-Za-z]+)", text)
if business_stage:
    business_stage = business_stage[0].strip()
else:
    business_stage = None


# 후속투자 추출
follow_up_investment = re.findall(r"후속투자 ([\d.]+)억원", text)
if follow_up_investment:
    follow_up_investment = follow_up_investment[0]

# 고용창출 (선정당시) 추출
job_creation_initial = re.findall(r"고용창출 \(선정당시\) (\d+)명", text)
if job_creation_initial:
    job_creation_initial = job_creation_initial[0]

# 고용창출 (최신) 추출
job_creation_latest_year= re.findall(r"고용창출 \(선정당시\) \d+명 -> \('(\d+)\) \d+명", text)
if job_creation_latest_year:
    job_creation_latest_year = job_creation_latest_year[0]

job_creation_latest = re.findall(r"고용창출 \(선정당시\) \d+명 -> \('\d+\) (\d+)명", text)
if job_creation_latest:
    job_creation_latest = job_creation_latest[0]

# 논문 및 특허 추출
paper_patent_count = re.findall(r"논문 및 특허 \((\w+)\) (\d+)건", text)
if paper_patent_count:
    paper_count = paper_patent_count[0][1]

# 매출 (‘20) 추출
revenue_2020 = re.findall(r"매출 \(‘20\) ([\d.]+)억원", text)
if revenue_2020:
    revenue_2020 = revenue_2020[0]

# 추출한 정보 출력
print("소재지:", location)
print("대표자 (생년):", founder)
print("사업화 단계:", business_stage)
print("후속투자:", follow_up_investment)
print("고용창출 (선정당시):", job_creation_initial, "명")
print("고용창출 (최신):", job_creation_latest_year,'년',job_creation_latest, "명")
print("논문 및 특허:", paper_count, "건")
print("매출 (‘20):", revenue_2020, "억원")

소재지: 경기화성시
대표자 (생년): 1973
사업화 단계: B
후속투자: 120.2
고용창출 (선정당시): 5 명
고용창출 (최신): 20 년 42 명
논문 및 특허: 29 건
매출 (‘20): 48 억원


# word


In [ ]:
import pandas as pd
import re

IR_KOR ='/content/drive/MyDrive/노동연구원/IR_KOR.docx'

from docx import Document

try:
    from xml.etree.cElementTree import XML
except ImportError:
    from xml.etree.ElementTree import XML
import zipfile

NAMESPACE = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}'
PARA = NAMESPACE + 'p'
TEXT = NAMESPACE + 't'

def get_docx_text(filename):
    document = zipfile.ZipFile(filename)
    xml_content = document.read('word/document.xml')
    document.close()
    tree = XML(xml_content)
    paragraphs = []
    for paragraph in tree.iter(PARA):
        texts = [node.text
                for node in paragraph.iter(TEXT)
                if node.text]
        if texts:
            paragraphs.append(''.join(texts))
    return '\n\n'.join(paragraphs)
    
# def extract_text_to_dataframe(filename):
#     extracted_text = get_docx_text(filename)
#     paragraphs = extracted_text.split('\n\n')
#     table_data = []
#     for paragraph in paragraphs:
#         row_data = paragraph.split('\n')
#         table_data.append(row_data)
#     df = pd.DataFrame(table_data)
#     return df

def extract_location(df):
    location = df[df.astype(str).apply(lambda x: x.str.contains('소재지', na=False)).any(axis=1)].iloc[:, 0]
    location = location.str.extract(r'([가-힣\s]+)').values.flatten()
    location = [loc.strip() for loc in location if loc.strip() != '소재지']
    location = ' '.join(location)
    return location

def extract_imformation(df):
  # 추출된 데이터 프레임 생성
    extracted_text = extract_text_to_dataframe(df)

    # 소재지 추출
    location = extract_location(extracted_text)

    # "소재지" 키워드 제거
    location = location.replace('소재지', '')
    location = location.replace('설립일', '')
    location = location.replace(' ', '')



    text = get_docx_text(df)


    # 대표자 (생년) 추출
    # 대표자 이름과 생년 추출
    founder_pattern = r"대표자 \(생년\)\s*(\S+)\s*\((\d{4})년\)"
    founder_match = re.search(founder_pattern, text)
    if founder_match:
        founder_name = founder_match.group(1)
        founder = founder_match.group(2)
    else:
        founder_name = None
        founder= None


    # 사업화 단계 추출
    business_stage = re.findall(r"시리즈\s*([A-Za-z]+)", text)
    if business_stage:
        business_stage = business_stage[0].strip()
    else:
        business_stage = None


    # 후속투자 추출
    follow_up_investment = re.findall(r"후속투자 ([\d.]+)억원", text)
    if follow_up_investment:
        follow_up_investment = follow_up_investment[0]

    # 고용창출 (선정당시) 추출
    job_creation_initial = re.findall(r"고용창출 \(선정당시\) (\d+)명", text)
    if job_creation_initial:
        job_creation_initial = job_creation_initial[0]

    # 고용창출 (최신) 추출
    job_creation_latest_year= re.findall(r"고용창출 \(선정당시\) \d+명 -> \('(\d+)\) \d+명", text)
    if job_creation_latest_year:
        job_creation_latest_year = job_creation_latest_year[0]

    job_creation_latest = re.findall(r"고용창출 \(선정당시\) \d+명 -> \('\d+\) (\d+)명", text)
    if job_creation_latest:
        job_creation_latest = job_creation_latest[0]

    # 논문 및 특허 추출
    paper_patent_count = re.findall(r"논문 및 특허 \((\w+)\) (\d+)건", text)
    if paper_patent_count:
        paper_count = paper_patent_count[0][1]

    # 매출 (‘20) 추출
    revenue_2020 = re.findall(r"매출 \(‘20\) ([\d.]+)억원", text)
    if revenue_2020:
        revenue_2020 = revenue_2020[0]

    # 추출한 정보 출력
    return print("소재지:", location),print("대표자 (생년):", founder), print("사업화 단계:", business_stage), print("후속투자:", follow_up_investment),print("고용창출 (선정당시):", job_creation_initial, "명"),print("고용창출 (최신):", job_creation_latest_year,'년',job_creation_latest, "명"),print("논문 및 특허:", paper_count, "건"),print("매출 (‘20):", revenue_2020, "억원")

All= extract_imformation(IR_KOR)
All


소재지: 경기화성시
대표자 (생년): 1973
사업화 단계: B
후속투자: 120.2
고용창출 (선정당시): 5 명
고용창출 (최신): 20 년 42 명
논문 및 특허: 29 건
매출 (‘20): 48 억원


(None, None, None, None, None, None, None, None)

In [ ]:
import re
import docx
docx_file ='/content/drive/MyDrive/노동연구원/teams_pdf/IR_KOR_1.docx'

# location,founder,business_stage,follow_up_investment,job_creation_initial,job_creation_latest_year,job_creation_latest,paper_count,revenue_2020


In [ ]:
extracted_text.head(60)

,0
0,엔트리움
1,설립일2013. 2. 1E-mailsnuven@ntrium.com팁스 선정월2013...
2,설립일2013. 2. 1
3,E-mailsnuven@ntrium.com
4,팁스 선정월2013. 8월
5,소재지경기 화성시
6,대표자 (생년) 정세영 (1973년)
7,Websitewww.ntrium.com
8,운영사명캡스톤파트너스
9,사업화 단계 시리즈 B : 시장확대


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import re

# IR_KOR ='/content/drive/MyDrive/노동연구원/teams_pdf/IR_KOR (1).docx'

from docx import Document

try:
    from xml.etree.cElementTree import XML
except ImportError:
    from xml.etree.ElementTree import XML
import zipfile

## 제목 250개 전부 뽑기


In [ ]:
from docx import Document

def extract_titles_and_contents(filename):
    doc = Document(filename)
    data = []
    
    for paragraph in doc.paragraphs:
        text = paragraph.text.strip()
        
        if text:
            if paragraph.style.name == 'Heading 1':  # 제목에 해당하는 스타일 이름 설정
                data.append({'title': text, 'content': ''})
            else:
                if data:  # 이전에 제목이 있었다면
                    data[-1]['content'] += text + '\n'
    
    return data

# 제목과 내용 추출
data = extract_titles_and_contents('/content/drive/MyDrive/노동연구원/teams_pdf/IR_KOR_1.docx')
i=0
# 추출된 제목과 내용 출력
for item in data:
  
  print(f"제목{i+1}:", item['title'])
  i+=1



제목1: 엔트리움
제목2: 프라센
제목3: 위브랩
제목4: 시옷플랫폼
제목5: 에이스틴
제목6: 노보믹스
제목7: 크레스프리
제목8: 다노
제목9: 키즈노트
제목10: 헬스브리즈
제목11: 라이클
제목12: 브레이브팝스컴퍼니
제목13: 포도코리아
제목14: 오비이랩
제목15: 드라마앤컴퍼니
제목16: 제이디랩
제목17: 큐리온코리아
제목18: 쿠쿠닥스
제목19: 스트라티오코리아
제목20: 스파코사
제목21: 채널코퍼레이션
제목22: 클라우다이크
제목23: 휴이노
제목24: 스몰머신즈
제목25: 다이닝코드
제목26: 비트파인더코리아
제목27: 브릿지모바일
제목28: 모션퀸
제목29: 유비파이
제목30: 알피이
제목31: 테그웨이
제목32: 브레인커머스
제목33: 이큐브랩
제목34: 레클
제목35: 요쿠스
제목36: 딥서치
제목37: 시어스랩
제목38: 에바인
제목39: 스튜디오씨드코리아
제목40: 인스에듀테인먼트
제목41: 엑센
제목42: 다비오
제목43: 리니어블
제목44: 룩시드랩스
제목45: 파이언스
제목46: 어반베이스
제목47: 디에스브로드캐스트
제목48: 아이엠랩
제목49: 뤼이드
제목50: 와이젯
제목51: 와인소프트
제목52: 비트루브
제목53: 오비츠코리아
제목54: 수아랩
제목55: 닷
제목56: 연시스템즈
제목57: 임프레시보코리아
제목58: 민코넷
제목59: 투블루
제목60: 더블미
제목61: 매드스퀘어
제목62: 데이블
제목63: 엠프로스
제목64: 아이알엠
제목65: 엔픽스
제목66: 제이커브
제목67: 스탠다임
제목68: 가우디오랩
제목69: 에너윈코리아
제목70: 매버릭
제목71: 나노미래생활
제목72: 비손콘텐츠
제목73: 인투코어테크놀로지
제목74: 브이터치
제목75: 밸런스히어로
제목76: 포그리트
제목77: 뉴로게이저
제목78: 아이오
제목79: 에버스핀
제목80: 뉴플로이
제목81: 플라즈맵
제목82: 디트로네
제목83: 스페클립스
제목84: 키페어
제목85: 크레셈
제목86: 아이엠지티
제목87: 하이리움산업
제목88: 아

In [ ]:
pip install PyPDF2


In [ ]:
# !pip install cffi==1.14.3
# !pip install chardet==3.0.4
# !pip install cryptography==3.2.1
# !pip install olefile==0.46
# !pip install pdfminer.six==20201018
# !pip install pycparser==2.20
# !pip install six==1.15.0
# !pip install sortedcontainers==2.2.2

In [ ]:
import zipfile
import sys

NAMESPACE = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}'
PARA = NAMESPACE + 'p'
TEXT = NAMESPACE + 't'


def get_docx_text(filename):
    document = zipfile.ZipFile(filename)
    xml_content = document.read('word/document.xml')
    document.close()
    tree = XML(xml_content)
    paragraphs = []
    for paragraph in tree.iter(PARA):
        texts = [node.text
                 for node in paragraph.iter(TEXT)
                 if node.text]
        if texts:
            paragraphs.append(''.join(texts))
    return '\n\n'.join(paragraphs)


# DOCX 파일로부터 텍스트 추출
docx_file = '/content/drive/MyDrive/노동연구원/teams_pdf/IR_KOR_1.docx'

result = get_docx_text(docx_file)

# 추출된 텍스트 출력
print(result)


엔트리움 

설립일2013. 2. 1E-mailsnuven@ntrium.com팁스 선정월2013. 8월소재지경기 화성시대표자 (생년) 정세영 (1973년)Websitewww.ntrium.com운영사명캡스톤파트너스사업화 단계 시리즈 B : 시장확대회사개요사 업 개 요분야전기ㆍ전자 - 디스플레이 - 디스플레이 부품 및 소재사 업 명비등방성 전도성 접착필름(ACF)용 도전성 입자 국산화사업내용나노융합소재 연구개발모바일기기, 반도체, 디스플레이 등 첨단 IT 제품용 핵심 나노소재 기술보유약물전달 시스템용 나노입자 개발 : 표적항암제 등 바이오 제약분야 기초기술 보유전기전도성 소재, 열전도성 소재, 전자파차폐 소재 개발 및 사업화비즈니스 모델수익창출 도전성 입자, EMI 차폐소재 등 납품을 통한 수익창출주요고객 SK하이닉스, UPAK(대만), SUNA(중국), 삼성전자, 앰코코리아, 애플, ISC(한국), TSE(한국), H&S하이텍(한국), 회명산업 등주요채널 국내외 고객사 직접 거래(B2B)핵심활동 도전성 입자 개발 및 이를 활용한 EMI 차폐소재 개발 및 납품 중 본딩 필름, 자외선 차단 필름 등 협업 개발 및 납품 중해외진출 대만 고객사에 공급 중이며, 중국, 미국, 유럽, 일본 등 제품 평가 중 16년 약 12만 달러, 17년 약 15만 달러, 18년 24만 달러 수출주요성과후속투자 120.2억원고용창출 (선정당시) 5명 -> ('20) 42명논문 및 특허 (특허) 29건매출 (‘20) 48억원기타 (수출) ('20) 8억원2014. 9. 소재부품기술개발사업 선정(투자자연계형-3년간 총 18억),2015. 1. 과제 연계 투자 35억 유치(산업은행, 캡스톤, SV인베스트)2016.12. 추가 투자 유치(기업은행 등 총 9.8억)2017. 3. SK하이닉스 기술혁신 기업 선정(소재 기업 최초 선정)2017. 7. 소재부품기술개발사업 신규 선정(3년간 총 21억)2017. 7. 과제 연계 투자 10.5억 유치(대경창투, SJ인베스트)2017. 7. 나노기술상용화 

In [ ]:
import pandas as pd

# 추출된 텍스트를 리스트로 분할
text_list = result.split('\n\n')

# 리스트를 DataFrame으로 변환
df = pd.DataFrame({'Text': text_list})

# DataFrame 출력
print(df[1000:1500])


                                                   Text
1000                                        • 서 비 스 사 진
1001                           팁스(TIPS) 창업기업 세부현황 D/B •
1002                                            사 업 개 요
1003                                            사 업 개 요
1004  설립일2013. 3. 25E-mailj@widevantage.com팁스 선정월201...
...                                                 ...
1495                                               주요성과
1496                                       매출 (‘21) 6억원
1497                         고용창출 (선정당시) 3명 -> (’21) 5명
1498                                          후속투자 10억원
1499                                   논문 및 특허 (특허) 12건

[500 rows x 1 columns]


In [ ]:
# import pandas as pd

# # 추출된 텍스트를 리스트로 분할
# text_list = result.split('\n\n')

# # 사업 개요가 시작되는 행 인덱스를 찾기
# page_break_indexes = []
# for i in range(len(text_list) - 1):
#     if text_list[i].strip() == '사 업 개 요' and text_list[i+1].strip() == '사 업 개 요':
#         page_break_indexes.append(i+1)

# # 페이지별로 데이터프레임 생성
# pages = []
# start_index = 0
# for end_index in page_break_indexes:
#     page_text = '\n\n'.join(text_list[start_index:end_index])
#     df = pd.DataFrame({'페이지': [page_text]})
#     pages.append(df)
#     start_index = end_index

# # 마지막 페이지 추가
# last_page_text = '\n\n'.join(text_list[start_index:])
# df = pd.DataFrame({'페이지': [last_page_text]})
# pages.append(df)

# # 페이지별 데이터프레임 출력
# for i, df in enumerate(pages):
#     print(f'페이지 {i+1}:')
#     print(df)
#     print('-' * 50)


In [ ]:
# import pandas as pd
# # pd.set_option('display.max_colwidth', None)

# # 추출된 텍스트를 리스트로 분할
# text_list = result.split('\n\n')

# # 페이지별로 데이터프레임 생성
# pages = []
# current_page = []
# for line in text_list:
#     if line.strip() == '팁스(TIPS) 창업기업 세부현황 D/B •' or line.strip() == 'TECH INCUBATOR PROGRAM FOR STARTUP':
#         if current_page:
#             df = pd.DataFrame({'페이지': ['\n\n'.join(current_page)]})
#             pages.append(df)
#             current_page = []
#     current_page.append(line)

# # 마지막 페이지 추가
# if current_page:
#     df = pd.DataFrame({'페이지': ['\n\n'.join(current_page)]})
#     pages.append(df)

# # 페이지별 데이터프레임 출력
# for i, df in enumerate(pages):
#     print(f'페이지 {i+1}:')
#     print(df)
#     print('-' * 50)


In [ ]:
import pandas as pd

# 추출된 텍스트를 리스트로 분할
text_list = result.split('\n\n')

# 페이지별로 데이터프레임 생성
pages = []
current_page = []
for line in text_list:
    if line.strip() == '팁스(TIPS) 창업기업 세부현황 D/B' or line.strip() == 'TECH INCUBATOR PROGRAM FOR STARTUP':
        if current_page:
            pages.append(current_page)
            current_page = []
    current_page.append(line)

# 마지막 페이지 추가
if current_page:
    pages.append(current_page)

# 페이지별 데이터프레임 출력
for i, page in enumerate(pages):
    df = pd.DataFrame({'페이지': page})
    print(f'페이지 {i+1}:')
    print(df)
    print('-' * 50)


페이지 1:
                                                  페이지
0                                               엔트리움 
1   설립일2013. 2. 1E-mailsnuven@ntrium.com팁스 선정월2013...
2                                       설립일2013. 2. 1
3                             E-mailsnuven@ntrium.com
4                                      팁스 선정월2013. 8월
..                                                ...
74                                   논문 및 특허 (특허) 11건
75                            기타 해외법인(미국) 설립(‘14.12월)
76                                     제 품• 서 비 스 사 진
77                                                제 품
78                                        • 서 비 스 사 진

[79 rows x 1 columns]
--------------------------------------------------
페이지 2:
                                                  페이지
0                  TECH INCUBATOR PROGRAM FOR STARTUP
1                                                위브랩 
2   설립일2013. 8. 6E-maildkim@wivlabs.com팁스 선정월2013....
3                                       설립일2013.

In [ ]:
df[63:64]


,페이지
63,설립일2013. 11. 6E-mailseenbio@gpcr.co.kr팁스 선정월20...


In [ ]:
import pandas as pd

# 추출된 텍스트를 리스트로 분할
text_list = result.split('\n\n')

# 페이지별로 데이터프레임 생성
pages = []
current_page = []
for line in text_list:
    if line.strip() == '팁스(TIPS) 창업기업 세부현황 D/B •' or line.strip() == 'TECH INCUBATOR PROGRAM FOR STARTUP':
        if current_page:
            pages.append(current_page)
            current_page = []
    current_page.append(line)

# 마지막 페이지 추가
if current_page:
    pages.append(current_page)

# 페이지별 데이터프레임 출력
for i, page in enumerate(pages):
    df = pd.DataFrame({'페이지': page})

    # 홀수 페이지에서 0, 1, 2행 삭제
    if (i + 1) % 2 == 1:
        df = df.iloc[4:]

    # 짝수 페이지에서 0, 1, 2, 3행 삭제
    if (i + 1) % 2 == 0:
        df = df.iloc[3:]

    print(f'페이지 {i+1}:')
    print(df)
    print('-' * 50)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
27       매출 (‘20) 4.6억원 논문 및 특허 (특허) 12건 후속투자 242.9억원
28                        고용창출 (선정당시) 3명 -> ('20) 48명
29                                기타 (수출) ('20) 4.6억원
30             제 품• 서 비 스 사 진사 업 개 요[악성표지인덱스 SW 예상 안]
31                                                제 품
32                                        • 서 비 스 사 진
33                                            사 업 개 요
--------------------------------------------------
페이지 110:
                                                  페이지
3                 대표자 (생년) 김기진 (1964년)설립일2011. 12. 19
4   Websitewww.3dmedivision.comE-mailceo@3dmedivis...
5             운영사명이노폴리스파트너스 (BNH인베스트먼트)팁스 선정월2016. 2월
6                         사업화 단계 시리즈 B : 시장확대소재지서울 중구
7                                                회사개요
8   사 업 개 요 분야바이오ㆍ의료 - 의료정보 및 시스템 - 기타 의료 정보 및 시스템...
9                                             사 업 개 요
10                                                 분야
11               바이오ㆍ의료 - 의료정보 및 시스템 - 기

# 최종
- 페이지별 자르기

In [894]:
import pandas as pd
import re
pd.set_option('display.width', None)


from docx import Document

try:
    from xml.etree.cElementTree import XML
except ImportError:
    from xml.etree.ElementTree import XML
import zipfile

NAMESPACE = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}'
PARA = NAMESPACE + 'p'
TEXT = NAMESPACE + 't'


def get_docx_text(filename):
    document = zipfile.ZipFile(filename)
    xml_content = document.read('word/document.xml')
    document.close()
    tree = XML(xml_content)
    paragraphs = []
    for paragraph in tree.iter(PARA):
        texts = [node.text
                 for node in paragraph.iter(TEXT)
                 if node.text]
        if texts:
            paragraphs.append(''.join(texts))
    return '\n\n'.join(paragraphs)


# DOCX 파일로부터 텍스트 추출
docx_file = '/content/drive/MyDrive/노동연구원/teams_pdf/1801-2038.docx'
result = get_docx_text(docx_file)
result[:1000]

'회사개요\n\n대표자 (생년)  허윤길(1993년)설립일2019.  8. 28\n\nWebsitewww.mayoube.co.krE-mailyg9676@mayoube.co.kr\n\n운영사명부산지역대학연합기술지주팁스 선정월2022. 6월\n\n사업화 단계  엔젤투자 단계 :  시제품 제작소재지부산 금정구\n\n사 업 개 요분야지식서비스 - 연구개발/엔지니어링서비스 - 기타연구개발/엔지니어링관련기술사 업 명머신러닝 기반 육아용품 맞춤추천 플랫폼 개발사업내용유해한 성분이 들어간 제품이 더욱 민감하게 다가오는 아기들에게 있어서 육아용품 구매 가장 중요한 기준이 성분, 그 성분들을 아무리 읽어봐도 이해할수 없   는 일반인 부모님들을 위해 아기에게 맞는 제품을 추천할 수 있도록 머신러닝  기반 육아용품 추천 서비스를 기획·개발 중이러한 기술을 스마트폰에 적용하여, 부모님들이 언제든지 내아기에게 맞는 제 품을 찾을수 있도록 체질, 특성, 성분의 역할 및 등급들을 맵핑하여 정량적인퍼센트로 제품의 적합도를 알려주는 것을 개발 중비즈니스 모델수익창출 성분이 우수한 제품들을 소싱하여 B2C형태로 판매주요고객 육아를 하는 부모님들중 성분에 대한 민감도를 가진 부모님들을 대상주요채널 안드로이드 마켓 ‘22.07월 런칭, iOS 버전 ’22.08월 런칭주요성과후속투자 6.6억원(신용보증기금, 엑센트리벤처스, 부산대학교기술지주, 부산지역연 합대학기술지주)고용창출 8명논문 및 특허 (특허) 상표국내등록 19건기타 비대면 육성사업 선정 (1억 6천만원)\n\n사 업 개 요\n\n분야\n\n지식서비스 - 연구개발/엔지니어링서비스 - 기타연구개발/엔지니어링관련기술\n\n사 업 명\n\n머신러닝 기반 육아용품 맞춤추천 플랫폼 개발\n\n사업내용\n\n유해한 성분이 들어간 제품이 더욱 민감하게 다가오는 아기들에게 있어서 육아\n\n용품 구매 가장 중요한 기준이 성분, 그 성분들을 아무리 읽어봐도 이해할수 없   는 일반인 부모님들을 위해 아기에게 맞는 제품을 추천할 수 있도록 머신러닝  기반 

In [895]:
# # 텍스트 파일 열기
# file_path = '/content/drive/MyDrive/노동연구원/teams_pdf/IR_KOR_ALL.txt'  # 파일 경로 지정
# with open(file_path, 'r') as file:
#     # 파일 내용 읽기
#     lines = file.readlines()

# # 줄 바꿈 제거 및 한 줄로 만들기
# result = "\n".join(line.strip() for line in lines)

# # # 각 줄 사이에 "\n\n" 추가
# # result = "\n\n".join(line.strip() for line in lines)

# # 결과 출력
# result[-2000:-1]

In [991]:
def extract_text_to_dataframe(result):
    text_list = result.split('\n\n')

    # 페이지별로 데이터프레임 생성
    pages = []
    current_page = []
    for line in text_list:
        if line.strip()=='팁스(TIPS) 창업팀 세부현황 D/B제 품 ㆍ 서 비 스 사' or line.strip()=='뉴로엑스티' or line.strip() == '경기 성남시(분당서울대병원 헬스케어혁신파크소재지팁스 선정월 2022. 3월사업화 단계 엔젤투자 : 시제품 검증팁스(TIPS) 창업팀 세부현황 D/B •' or line.strip() == '팁스(TIPS) 창업팀 세부현황 D/B •' or line.strip() == 'ECH INCUBATOR PROGRAM FOR STARTUP' or line.strip() =='⦁원격 피드백' or line.strip() == '지피씨알' or  line.strip() =='팁스(TIPS) 창업기업 세부현황 D/B •' or line.strip() =='TECH  INCUBATOR  PROGRAM  FOR   STARTUP' or line.strip() == 'TECH INCUBATOR PROGRAM FOR STARTUP' or line.strip() =='제 품 ㆍ 서 비 스 사 진TECH INCUBATOR PROGRAM FOR STARTUP' or line.strip() =='제 품 ㆍ 서 비 스 사 진팁스(TIPS) 창업기업 세부현황 D/B •':
            if current_page:
                pages.append(current_page)
                current_page = []
        current_page.append(line)

    # 마지막 페이지 추가
    if current_page:
        pages.append(current_page)

    dfs =[]
    dfs_text =[]

    # 페이지별 데이터프레임 출력
    for i, page in enumerate(pages):
        df = pd.DataFrame({'페이지': page})

        # # 홀수 페이지에서 0, 1, 2행 삭제
        # if (i + 1) % 2 == 1:
        #     df = df.iloc[4:]

        # # 짝수 페이지에서 0, 1, 2, 3행 삭제
        # if (i + 1) % 2 == 0:
        #     df = df.iloc[3:]

        # print(f'페이지 {i+1}:')
        # print(df)
        # print('-' * 50)
        dfs.append(df)

      # 행을 저장할 리스트
        result_list = []

      # 데이터 프레임의 행을 반복하면서 각 행을 리스트로 변환하여 결과 리스트에 추가
        for index, row in df.iterrows():
            row_list = row.values.tolist()
            result_list.extend(row_list)
        dfs_text.append(result_list)

        # All=extract_imformation(df)
        
        # print(f'페이지 {i+1}:')
        # print(All)
        # print('-' * 50)
    return dfs ,dfs_text


In [983]:
pd.set_option('display.width', None)

A=extract_text_to_dataframe(result)
dfs =A[0]

In [984]:
len(dfs)


239

In [997]:
A=extract_text_to_dataframe(result)
dfs_text=A[1]

In [1001]:
for i in range(238):
  dfs_text[i]=[value for value in dfs_text[i] if len(value.strip("'")) <= 400]

## 정보 추출하기

In [1005]:
#dfs사용
def extract_location(df):
    # location = df[df.astype(str).apply(lambda x: x.str.contains('소재지', na=False)).any(axis=1)].iloc[:, 0]
    # location = location.str.extract(r'([가-힣\s]+)').values.flatten()
    # location = [loc.strip() for loc in location if loc.strip() != '소재지']
    # location = ' '.join(location)
    row_index = df[df['페이지'].str.contains('소재지')].index[0]
    location = df.iloc[row_index, 0].split('소재지')[-1].strip()
    if location.find('대표자'):
      start_index = location.find('대표자')
      location = location[:start_index]

    # Remove the portion starting from "대표자"
    
    return location



def extract_imformation(dfs, dfs_text):
    # 추출된 데이터 프레임 생성
    extracted_text = dfs
    # 소재지 추출
    location = extract_location(extracted_text)
    

    # # "소재지" 키워드 제거
    # location = location.replace('소재지', '')
    # location = location.replace('설립일', '')
    # location = location.replace(' ', '')


    #dfs_text사용
    text = f'{dfs_text}'


    # 대표자 (생년) 추출
    # 대표자 이름과 생년 추출
    founder_pattern = r"대표자 \(생년\)\s*(\S+)\s*\((\d{4})년\)" 
    founder_match = re.search(founder_pattern, text)
    if founder_match:
        founder_name = founder_match.group(1)
        founder = founder_match.group(2)
    else:
        founder_name = None
        founder= None


    # 사업화 단계 추출
    business_stage = re.findall(r"(시리즈\s*[A-Za-z]+|엔젤투자)", text)
    if business_stage:
        business_stage = business_stage[0].strip()
    else:
        business_stage = None


    # 후속투자 추출
    follow_up_investment = re.findall(r"후속투자 ([\d.]+)억원", text)
    match = re.search(r"후속투자\s+(\d+)억원", text) or re.search(r"후속투자\s\s\s+(\d+)억원", text) or re.search(r"(\d+)억원\s+\(선정당시\)", text)
    
    if follow_up_investment:
      follow_up_investment = follow_up_investment[0]
    else :
      follow_up_investment =None
    if match:
      follow_up_investment = match.group(1)
    else :
      follow_up_investment =None

    # 고용창출 (선정당시) 추출
    job_creation_initial = re.findall(r"고용창출 \(선정당시\) (\d+)명", text) 
    match = re.search(r'고용창출 (\d+)', text) or re.search(r"고용창출\s+\(선정당시\)\s+(\d+)명", text) or re.search(r"\((?:선정당시)\)\s+(\d+)명", text) or re.search(r"고용창출\s+(\d+)명", text) or re.search(r"고용창출\s+(.*?)명", text)
    if job_creation_initial:
        job_creation_initial = job_creation_initial[0]  
    else:
      if match :
        job_creation_initial= match.group(1)
      else : 

        # 숫자 추출을 위한 정규표현식 패턴
        pattern = r'(\d+)명'

        # 숫자 추출
        job_creation_initial = None
        for item in dfs_text:
            match = re.search(pattern, item)
            if match:
                job_creation_initial =match.group(1)
                break




    # 고용창출 (최신) 추출
    pattern_list = [r"\('20\) (\d+)명", r"\(‘21\) (\d+)명", r"\(‘20\) (\d+)명", r"\(’21\) (\d+)명",r"\('21\) (\d+)명",r"\('21\)  (\d+)명",r"\('20\)  (\d+)명"]

    latest = None
    for item in dfs_text:
        for pattern in pattern_list:
            match = re.search(pattern, item)
            if match:
                latest = match.group()
                break
        if latest:
          
            break
   
    # match = re.search(r'고용창출 (\d+)논문', ㅑㅅㄷ)

    # if match:
    #   latest = match.group(1)

    

    # 논문 및 특허 추출
    paper_patent_count = re.findall(r"논문 및 특허 \((\w+)\) (\d+)건", text) #논문 및 특허 (특허)  11건
    if paper_patent_count:
        paper_count = paper_patent_count[0][1]
    else :
      paper_count =None

      match = re.search(r"\((\w+)\)(\d+)건", text) or  re.search(r"\((\w+)\)\s\s(\d+)건", text) 
      if match:
          paper_count = match.group(2)
      else :
        match = re.search(r"\(특허\)\s+(\d+)건", text) or re.search(r"\(논문\)\s+(\S+\s+\S+)\s+(\d+)건", text)
        if match:
          paper_count = match.group()

        else :
          match = re.search(r"논문 및 특허(.*?)기타", text, re.DOTALL)   
          if match:
              paper_count = match.group(1).strip()
          else : None
              # print(content)
      
    # if paper_count :
    #   if paper_count.find('기타'):
    #       paper_count = re.sub(r'기타.*', '', text)

    

    # 매출 추출
    pattern_list2 =[r"매출 \(‘20\) ([\d.]+)억원" , r"매출 \(‘21\) ([\d.]+)억원",r"매출 \ ('20\) ([\d.]+)억원) ",r"매출 \(’21\) ([\d.]+)억원",r"매출 \(‘21\)  ([\d.]+)억원"] 

    revenue_2020 = None
    for item in dfs_text:
      for pattern in pattern_list2:
        match = re.search(pattern, item)
        if match:
            revenue_2020 = match.group()
            revenue_2020 =revenue_2020[3:]

            break
        else :
          revenue_2020 = None
    match = re.search(r"\((?:‘20)\)\s+(\d+억원)", text)
    if match:
      revenue_2020 = match.group()

    else :
      revenue_2020 =None


    

    # print("0소재지:", location)
    # print("1대표자 (생년):", founder)
    # print("2사업화 단계:", business_stage)
    # print("3후속투자:", follow_up_investment)
    # print("4고용창출 (선정당시):", job_creation_initial, "명")
    # print("5고용창출 (최신):",latest)
    # print("6논문 및 특허:", paper_count, "건")
    # print("7매출 :", revenue_2020)

    # 추출한 정보 출력
    return location,founder,business_stage,follow_up_investment,job_creation_initial,latest,paper_count,revenue_2020



In [1007]:
# A_138 =('경기 성남시(분당서울대병원 헬스케어혁신파크)', 'None', '엔젤투자', '약 2억원(21년, 주식회사 액트너랩), 개인투자자 허윤석 외 8인(약 1억원)','4','None','특허출원 5건, 상표 1건, 저작권 1권 출원','None')
All = []

for i in range(238):
  A =extract_imformation(dfs[i], dfs_text[i])
  # print(A)
  All.append(A)
# All.append(A_138)

IR_1801_2038= pd.DataFrame(All, columns=['소재지','대표자 (생년)', '사업화 단계','후속투자','고용창출 (선정당시)','고용창출 (최신)','논문 및 특허','매출'])

In [1009]:
IR_1801_2038.to_excel('/content/drive/MyDrive/노동연구원/teams_pdf/IR_1801_2038.xlsx', index=False)

In [1008]:
IR_1801_2038


,소재지,대표자 (생년),사업화 단계,후속투자,고용창출 (선정당시),고용창출 (최신),논문 및 특허,매출
0,부산 금정,1993,엔젤투자,None,8,None,"(특허) 상표국내등록 19건', '",None
1,서울 서초,None,None,None,5,None,"특허 출원 준비중, 상표국내출원 1건', '",None
2,경기 용인,1980,엔젤투자,None,8,None,None,None
3,서울 성동,1976,엔젤투자,None,3,None,None,None
4,강원 원주,1988,None,None,7,None,"국내출원 3건, 국내등록 2건, 상표국내등록 1건', '",None
5,서울 동대문,None,시리즈 A,None,12,None,None,None
6,경북 포,1975,엔젤투자,None,4,None,None,None
7,서울 강남,1986,엔젤투자,None,청년인력 중심으로 현재 4,None,None,None
8,인천 연수,1987,None,None,5,None,None,None
9,서울 마포,1980,None,None,12,None,"(특허) 국내 출원 2건, 국내 등록 1건, 해외 출원 1건 (상표) 국내출원 2건...",None
